# Danish Tourism Patterns - Analysis Notebook

## 1. Motivation

### What is your dataset?

We're analyzing Danish tourism patterns using data from multiple sources, which all are provided below:

1. **Danish Tourism Statistics**, (Statistics Denmark - Statistik Banken)
   - Demographics data: Socioeconomic (FU14), Regional (FU17), Age (FU18): https://www.statistikbanken.dk/statbank5a/SelectVarVal/Define.asp?MainTable=FU14&PLanguage=0&PXSId=0&wsid=cftree
   - Transport data: Modes of transportation (FF3), Departing Passengers (FLYV92): https://www.statistikbanken.dk/ff3
   - Destination data: Vacations per country (FF1): https://www.statistikbanken.dk/statbank5a/SelectVarVal/Define.asp?MainTable=FF1&PLanguage=0&PXSId=0&wsid=cftree

2. **Air Travel Comparisons**, (World Bank Group)
   - DataBank tourism data: https://databank.worldbank.org/id/7ad54403

3. **Global Reference Data**
   - TripAdvisor Restaurants (31 European cities): https://www.kaggle.com/datasets/damienbeneschi/krakow-ta-restaurans-data-raw/code
   - Global Peace Index (GPI): https://www.kaggle.com/datasets/natalyreguer in/global-peace-index-gpi
   - Cost of Living Index: https://www.numbeo.com/cost-of-living/rankings_by_country.jsp
   - Mean Temperature by Country: https://www.kaggle.com/datasets/palinatx/mean-temperature-for-countries-by-year-2014-2022?select=combined_temperature.csv
   - CO2 Emissions: https://flightemissionmap.org/#Copenhagen/55.67,12.57/127/20000

4. **Image- og potentialeanalyse 2024**, (VisitDenmark)
   - Tourism data: Activities on vacations (page 19), Reasons for vacation abroad (page 21): https://www.visitdenmark.dk/sites/visitdenmark.com/files/2024-07/Image%20og%20potentialeanalyse%20for%20Danmark%202024.pdf

The Danish data covers demographics with detailed stats on who travels and how much they spend. The global data helps understand why Danes choose certain destinations.

### Why did you choose these datasets?

We could not find a unified datasets that did not restrict the analysis, as a result we choose several datasets. We wanted to answer questions like:
- How has Danish tourism changed over time?
- What factors make destinations popular?
- Do different demographics travel differently?
- Why do Danes travel abroad?
- Which modes of transportation are most used?
- And just in general get as much informations about travelling, with focus on danes.

We combined government statistics with global data to get both local patterns and international context. The idea was to create something that helps people understand Danish travel trends and potentially plan their own trips.

### What was your goal for the end user's experience?

Our goal for the end user is that they are able to see how Danish tourism has evolved, compare countries on different metrics, explore demographic patterns and in the end be able to make informed travel decisions, and find out where their next travel destination should be.

## 2. Basic stats

### Data cleaning process

Each visualization needed different cleaning approaches:

<details>
  <summary><b>World Bank Group</b></summary>
The dataset is sourced from the World Bank Group's Data Bank. The platform allows to choose features and time period and we decided using the entire period for available data, which was from 1996 to 2019, and the following columns:
  <ul>
    <li>Total population</li>
    <li>Number of international tourism departures</li>
    <li>Per capita departures (a derived metric from the two features above)</li>
    <li>GDP (Purchasing Power Parity, current international dollars)</li>
    <li>Country name</li>
  </ul>

Before we could create the visualization, we needed to clean and prepare the data. This included removing the thousand separators (commas) from the numeric fields, converting these fields to proper numeric formats, and handling any missing values. We also added a new column to categorize each country by continent so that the bubbles in the chart could be colored accordingly. We also normalized most indicators to a 0–100 scale for consistent color mapping in the visualization. 

We were aware that the dataset was incomplete and that some countries were missing data for certain years, which would cause their bubbles to appear or disappear in the chart. However, we decided this was acceptable. Removing these countries entirely would have made the chart less representative of global patterns. Our main goal was to show the general trend that higher GDP tends to be associated with a higher number of international tourism departures—not to track every country's behavior in detail. The only country we needed complete data for was Denmark, and that data was available.

```python
import pandas as pd
import numpy as np

# Load World Bank tourism data
df = pd.read_csv('../data/bubble_plot.csv')

# Clean currency and numeric fields
df['GDP'] = pd.to_numeric(df['GDP, PPP (current international $) [NY.GDP.MKTP.PP.CD]'].str.replace(',', ''), errors='coerce')
df['Population'] = pd.to_numeric(df['Population, total [SP.POP.TOTL]'], errors='coerce')
df['Departures'] = pd.to_numeric(df['International tourism, number of departures [ST.INT.DPRT]'].str.replace(',', ''), errors='coerce')
df['PerCapita'] = pd.to_numeric(df['International Tourism Departures per capita'], errors='coerce')

# Add continent categorization
continent_mapping = {
    'Denmark': 'Europe', 'Sweden': 'Europe', 'Norway': 'Europe', 
    'United Kingdom': 'Europe', 'Germany': 'Europe', 'France': 'Europe',
    'United States': 'North America', 'Canada': 'North America',
    'China': 'Asia', 'Japan': 'Asia', 'India': 'Asia',
    'Australia': 'Oceania', 'New Zealand': 'Oceania',
    'South Africa': 'Africa', 'Egypt, Arab Rep.': 'Africa',
    'Brazil': 'South America', 'Argentina': 'South America',
    'Saudi Arabia': 'Middle East', 'United Arab Emirates': 'Middle East'
}

df['Continent'] = df['Country Name'].map(continent_mapping).fillna('Other')
```
</details>


<details>
  <summary><b>For the choropleth map</b></summary>

```python
import folium
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Load all datasets
cost_df = pd.read_csv("../data/CostOfLiving.csv")
gpi_df = pd.read_csv("../data/GPI.csv")
temp_df = pd.read_csv("../data/combined_temperature.csv")
restaurants_df = pd.read_csv("../data/TA_restaurants_europa.csv")
CO2_df = pd.read_csv("../data/CO2Emission.csv")

# Clean and standardize country names
cost_df.columns = cost_df.columns.str.strip()
cost_df['Country'] = cost_df['Country'].str.strip()

# Country name mapping for standardization
country_name_map = {
    'United States': 'United States of America',
    'Russia': 'Russian Federation',
    'South Korea': 'Korea, Republic of',
    'Bosnia And Herzegovina': 'Bosnia and Herzegovina',
    'United Kingdom': 'United Kingdom'
    # ... (extensive mapping)
}
```
</details>

<details>
  <summary><b>For the demographics data</b></summary>

```python
# Process socioeconomic spending data
socio_translations = {
    "Gennemsnitshusstand": "Average Household",
    "Selvstændig": "Self-employed",
    "Lønmodtager på højeste niveau": "High Income",
    "Lønmodtager på mellemniveau": "Medium Income",
    "Lønmodtager på grundniveau": "Basic Income",
    "Arbejdsløs": "Unemployed",
    "Uddannelsessøgende": "Student",
    "Pensionist": "Pensioner",
    "Ude af erhverv We øvrigt": "Not in Workforce"
}

socio_df = socioeconomic.copy()
socio_df['Group_EN'] = socio_df['Group'].map(socio_translations)
socio_df['Total'] = socio_df['Packages'] + socio_df['Restaurants'] + socio_df['Accommodation']
```
</details>


<details>
  <summary><b>VisitDenmark</b></summary>
The dataset about reasons for spending the holiday abroad was sourced from VisitDenmark, a Danish organization, so the original data was in Danish. Only minimal preprocessing was needed. This included translating the category names from Danish to English and converting proportions into percentages by multiplying them by 100, which was necessary for use in the waffle chart. There were no missing values which made it easier. 

The dataset regarding activities on vacations was also sourced from VisitDenmark. Again it only required minimal preprocessing: The category names were translated from Danish to English and the values were normalized. 
</details>

<details>
  <summary><b>Statistics Denmark</b></summary>
The dataset regarding most popular destinations was found through Statistics Denmark. The country names were translated from Danish to English and mapped to their respective continents as well. Missing values were marked with "..", which we replaced with 0, to keep the numeric format. The formatting of the year values was also corrected. We chose to use data from 2023 only, as it was the most complete, and we already had other visualizations showing changes over time through animations.

The dataset about the number of departing passengers from Danish airports was likewise sourced from Statistics Denmark. The date column looked like this "2001M04", which required formatting. We split this into separate year and month values, and set the day to 1 since there was only one observation per month. The final result of this formatting was converted to a proper datetime format. By now it was possible to create the line plot. The diverging bar plot additionally required finding the mean of the values and subtracting this from all observations. 

The dataset about the modes of transporation was sourced from Statistics Denmark. Once again, the category names were translated from Danish to English and missing values were marked with "..", which we replaced with 0, to keep the numeric format. We decided to focus on holidays of at least 4 nights or more - we found this more representative, especially for a comparison across domestic and international travels. We decided to drop two transporations categories, bike and sailboat, as these had such small values compared to the other modes of transporation that it did not really make sense. 
</details>

### Dataset statistics

| Visualization | Dataset | Size | Coverage | 
|--------------|---------|------|----------|
| Bubble Chart | World Bank Group | 5,000+ rows | 1996-2019, 200+ countries |
| Choropleth | Global Reference Data | 124,000+ rows | All from cost of living, peace, rent index, temperature to groceries prices CO2, resturant prices and also tripadvisor informations about best style resturants |
| Radar Charts | Statistics Denmark | 30 rows | 2023, who spends money on travelling |
| Vertical Stacked Bar Chart | Statistics Denmark | 122 rows | 2017-2023, total of 37.308.735 vacations recorded|
| Line Chart & Diverging Bar Chart | Statistics Denmark | 288 rows | 2001-2024, all Danish commercial airports |
| Treemap | Statistics Denmark | 49 rows | 2001-2023, top 50 most popular destinations every year|
| Waffle Chart | VisitDenmark | 11 rows | 2023, survey answers|
| Horizontal Stacked Bar Chart | VisitDenmark | 11 rows | 2023, survey answers|




### Key findings from exploratory data analysis
- Danish tourism departures per Capita grew from 0.957 to 1.563 (1996-2019).
- Cost of living has stronger correlation with destination choice than safety.
- High income groups spend 3x more on package holidays.
- Danes prefer flying internationally and drive when travelling within Denmark.
- The COVID-19 virus had a huge impact on the airline industry.
- Danes usually spend most of their holiday during the summer months. 
- The countries in south Europe are popular tourism destination for Danes. 
- When travelling abroad Danes use more time on cultural activities while leisure activities are popular when vacationing in Denmark.
- Danes have many reason for vacationing abroad. 
- A trend of "higher GDP per capita, more international tourism departures" exist. 
- The CO2 expendature is almost linearly related to the distance to the destination.

## 3. Data Analysis

### Describe your data analysis and explain what you've learned about the dataset

---
#### Denmark's Tourism Journey - Bubble Chart

In this analysis, we explored tourism trends in relation to economic development across countries, with a particular focus on Denmark. We hypothesized that the number of international tourism departures had a positive correlation with GDP per capita across time. To explore these relationships over time, we chose to create a bubble chart, as it effectively illustrates changes and comparisons across multiple dimensions. The bubble chart visualization tracks Denmark's tourism evolution compared to other countries and shows the general trend we hypothesized. 

**Key Insights:**
- There is a general trend where higher GDP per capita is associated with more international tourism per capita. This supports the hypothesis that wealthier populations travel abroad more frequently.
- Denmark consistently ranks among the higher end of tourism per capita, particularly relative to countries with similar or even higher GDP per capita. This suggests a strong inclination toward international travel.
- European and North American countries tend to cluster in the upper-right quadrant (high GDP and high tourism), while countries from Africa and parts of Asia remain lower in both metrics.
- Nordic countries show similar patterns, but Denmark leads regionally.
- Over the years, many countries show an upward movement in GDP per capita, and this is often resulting in a likewise growth in number of international departures. 

---
#### Global Destination Factors - Choropleth Map

The choropleth map is an interactive world map analyzing eight different destination factors:
- Cost of Living Index
- Global Peace Index
- Rent Index
- Temperature (celsius)
- Groceries Index
- CO2 Emissions (kg)
- Restaurant Prices
- Restaurant Information
The preprocessing steps are already described briefly in the earlier section. Overall, all the data had to be merged correctly into a master dataframe before it was possible to build a Folium map. Each layer in the map corresponds to one of the factors and the user can switch between these different views. 

**Key Insights:**
- Living condition indicators vary significantly across Europe, with Northern and Western European countries generally showing higher living and rent costs, but also higher peace ratings.
- The CO2 expendature is almost linearly related to the distance to the destination.
- Restaurant data emphasizes that the restaurant cost is not always directly related to the restaurant ratings. This must be taken with a grna of salt as this is extremely aggregated data. 
- Since the temperature was a reason for leaving Denmark this view would probably be useful if Danes used the visualization as a tool for planning vacations. 

---
#### Demographic Spending Patterns - Radar Charts

MISSING: Three radar charts compare spendings across demographics, being socioeconomic class, age and region. They are all related to the three travel expenses restaurants, package holidays and accomodation. 

**Key Insights:**
- Self-employed heavily favor package holidays
- Students spend proportionally more on accommodation
- Regional differences show Copenhagen has up to 40% higher spending within the chosen categories. 

---
#### Correlation Analysis - Destination Factors

MISSING: The correlation matrix reveals relationships between the metrics

**Key Insights:**
- Cost metrics highly correlated (0.9+ between most)
- Cost vs Safety: -0.524 (moderate negative)
- Temperature vs CO2: +0.512 (climate impact)
- Cost vs Temperature: -0.317 (warmer = cheaper)

We also used:
- Correlation analysis to identify destination attractors
- Time series analysis for tourism trends
- Normalization techniques for cross-metric comparison
- Geographic clustering through choropleth visualization

---
#### Vertical Stacked Bar Plot - Modes of transport
We analyzed Danish vacation transport patterns for the year 2023, distinguishing between domestic and international travel. The data was cleaned as explained above and visualized using Bokeh. 
**Key Insights:**
- Danes primarily use cars for domestic vacations, possibly due to the country's compact geography, high car ownership and strong road infrastructure.
- Airplanes dominate the international travel scene. 
- Air travel had consistently 3 million trips per year except for COVID-19 years. Air travel dropped significantly in 2020–2021, while domestic car travel surged, highlighting the effect of the COVID-19 pandemics travel restrictions. 

---
#### Line plot - Departing Passengers
We used the data on number of departing passengers from Danish airports in two ways. For this plot it was treated as a time series and plotted as a line plot. We were especially interested in finding events which's impacts were reflected in the data. 
**Key Insights:**
- The number of passengers departing from Danish airports has generally increased
- Air travel nearly collapsed in the beginning of 2020 as a result of the COVID-19 pandemic.
- External economic factors such as the Great Recession also affects the airline industry.

---
#### Diverging bar plot - Departing Passengers per month
The other way we used the data for number of departing passengers from Danish airports was by aggregating it by taking the mean across months. With this method we gained a 20-year mean for each month illustrating the seasonal changes of air travel. 
**Key Insights:**
- It appears that Danes mostly fly during the summer months. This does make sense as it coincides with the vacation season. 
- We had hypothesized that other months—such as February, due to winter break in week 7, or December, due to holiday travel to visit family—would show increased travel activity. However, these trends were not clearly evident in the data or the visualization.
---
#### Waffel Chart - Reasons for holidays abroad
The data from VisitDenmark was very important as it gave insights into the more qualitative aspects of the Danish travel patterns. Among these were the reasons for travelling abroad compared to staying in Denmark. The data was analyzed with different plots but we ended up deciding on the waffel chart as it provides an overview and is easy to compare sizes of the different categories.  
**Key Insights:**
- Danes are actively choosing what the world beyond has to offer both culturally and nature-wise.
- Some Danes see limitations in the Danish weather and the feeling of everyday-life. 

---
#### Treemap - Popular Destinations
The data from Statistics Denmark was cleaned and we selected 2023 as it offered the most complete dataset. We chose not to include another map-based visualization, as it was already well represented by our comprehensive interactive "tool" map. However, we still needed a way to highlight the hierarchical structure of travel destinations — particularly the preference for Europe and southern countries. A treemap was well suited for this purpose, and Plotly provided much of the functionality required to create it.

**Key Insights:**
- Clear preference towards Europe
- Spain is the unrivaled favourite!
- Asia is the second most visited continent by Danes. 
---
#### Horizontal Stacked Bar Chart - Activities
The data from VisitDenmark for this visualization was more descriptive, as people were asked about their typical vacation activities. It made sense to approach this in terms of proportions, so we decided to use two horizontal stacked bar charts to allow for easy comparisons between vacation activities abroad and those in Denmark.

**Key Insights:**
- Danes appear more culturally engaged when abroad. 
- Leisure and entertainment-focused activities are more common when vacationing in Denmark.
---

## 4. Genre. 

### Which genre of data story did you use?

For this project, we chose a magazine-style approach with interactive elements. This genre works well because it combines guided storytelling with user exploration, which is perfect for a topic like tourism where people have different interests and questions. Additionally, the website was designed such that we were able to use scrollytelling. Scrollytelling is a narrative technique where elements appear or change as the reader scrolls, which let's us guide users through the visualization and present the findings in the most natural order.

### Which tools did you use from each of the 3 categories of Visual Narrative (Figure 7 in Segal and Heer). Why?
We incorporated a variety of tools from all three categories of the Visual Narrative framework from Figure 7. These will be discussed below:

**Visual Structuring**
- *Establishing shot*. Our website lacks a strong establishing shot in the traditional sense. It does not explicitly guide the user’s attention upon first view. This is likely because of our consistent (but restrictive) design and layout across the site. One could argue that instead of a single establishing shot, we guide users using other techniques like interactivity, scrollytelling and visual consistency.
 
- *Consistent Visual Platform* is one of the most prominently used techniques on our site. For example, the radar plot allows users to switch between categories while keeping the visual layout intact. Similarly, the interactive map changes content when different categories are selected, but its styling remains consistent. The vertical stacked bar chart comparing travel abroad vs. travel within Denmark also maintains consistent visual encoding across the datasets. This uniformity ensures users stay oriented and find transitions intuitive and visually coherent.

- *Progress bar* is not used much throughout our visualization but the animated bubble chart includes a timeline slider, essentially functioning as a progress bar. This allows users to control the temporal dimension and explore trends year by year, especially if they are interested in a specific time point.

- *Checklist structures* appear in the interactive map, where users can see available categories and revisit previous ones. This helps users understand the scope of the content and track what they've already explored.

**Highlighting**
- *Close-Ups* exist in several of the visualization, such as the map, treemap, and bubble chart, as they are interactive by standard. This allows users to zoom in or click for detailed information which supports deeper exploration and personalization of the data experience. The functionalities discussed here also covers the *Zooming* tool. 
- *Feature Distinction* refers to how we visually separate or emphasize key data elements. It is primarily done using color and/or size and is both used on a visualization level and on a website level.
- *Character Direction* IDK
- *Motion* IDK
- *Zooming* Mentioned above
- *Additional highlights* includes the change of background colour in our line plot. Here the COVID-19 period is highlighted to explain and emphasize the reason for the sudden drop in the graph. In the bubble plot we track Denmark's position throughtout the years aligning with our focus on Denmark.

**Transition guide**
- *Viewer (Camera) Motion* is relevant in relation to our scrollytelling where the view moves through the visual space. As mentioned before this enables us to guide the user through our visualization. 
- *Object Continuity* is extremely incorporated into our website: We maintain structural consistency across all web pages through object continuity. For instance, each page starts with a recognizable setup — a title, a key image, and boxes with key insights. This repetition reinforces user expectations and aids navigation.
- *Animated Transitions* is used in the animated bubble chart where we incorporates transitions between years, simulating a stop-motion effect. This makes temporal changes more intuitive and visually engaging, helping users perceive trends as continuous and natural.

### Which tools did you use from each of the 3 categories of Narrative Structure (Figure 7 in Segal and Heer). Why?

**Ordering**
- *User-Directed*: Our website structure is user-directed. Visitors are free to select among several different pages, each focusing on a specific thematic area. However, this freedom is still framed within a semi-linear flow — users are not navigating randomly, but through a list of themes ordered accordingly.

- *Linear*: Once a user selects a page, the structure shifts toward a linear, author-driven narrative, via scrollytelling. As users scroll down the page, visualizations and text are revealed in a pre-defined sequence. This allows us to guide users through the "story", where each step builds on the previous one.

- We would argue that this hybrid approach balances exploration and narrative control: The user chooses what to explore, and we guide them through the content once inside.

**Interactivity**
- *Hover Highlighting / Details* is a core interactive feature across most of our visualizations. For example, users can hover over elements and graphs to obtain extra information. This enables the user to understand the specific data point and does not require the user to be skilled at reading graph values.

- *Filtering / Selection / Search* plays an important role in simplifying complexity. In visualizations such as the radar plot or interactive map, users can select specific categories or groups. This allows them to focus on what's relevant, enhancing personalization and interpretability.

- *Navigation Buttons* have been limited to a minimum ensuring the user only transition to new pages when needed and preserving a clean layout. In visualizations, where buttons are necessary (e.g. to change views or datasets), they are clearly labeled and placed at the top of the visuals to avoid disrupting the user’s flow.

- *Explicit Instruction* is offered for more complex plots such as the vertical stacked bar plot where the user can shift between datasets or the interactive map where the toggle functionality is explained. 

- *Stimulating Default Views*. We made thoughtful choices in what is displayed by default in the visualizations with several views. In the interactive map, we default to Cost of Living, a topic of wide interest. In the radar plot, we show spending habits across socioeconomic groups, anticipating user curiosity around inequality and financial priorities. These default views act as strong entry points, inviting users into the data without overwhelming them.

**Stimulating**
- *Captions / Headlines* Each section of our pages is accompanied by headings and subheadings that explain the purpose of the visual or section. These headlines help orient users and convey the main message quickly. We have sometimes used alliteration and titles that rhymes just to make it more engaging to read. 

- *Annotations* are included to draw attention to key data points or explain anomalies. For example, highlighting the COVID-19 period in the line plot. This guides interpretation and emphasizes important trends.

- *Accompanying Article* is naturally integrates via our magazine style format as narrative text are placed alongside visual. This is critical for tying visual evidence to real-world implications and for storytelling.

- *Multi-Messaging* happens in our website by combining headlines, tooltips, annotations, and visual design resulting in communication on multiple layers of meaning. For example, users can understand trends at a glance and dive deeper with hover features or narrative text.

- *Introductory Text* is what most of the pages begin with. It sets the context and the stage for the data. This includes background information, the main theme being explored, and what the user should expect.
- *Summary / Synthesis* are provided as conclusion boxes at the end of some visual sections, summarising insights and helping users understand the bigger picture.

## 5. Visualizations

We selected four main visualizations that work together to tell the story of Danish tourism from different angles.

The bubble chart shows Denmark's tourism journey from 1996 to 2019, with GDP on a log scale on the x-axis and tourism departures per capita on the y-axis. The bubble size represents population, and the animation progresses year by year. We marked Denmark with a star to make it easy to follow throughout the animation. This visualization was chosen because it shows Denmark's relative position in global tourism evolution - you can see how we've grown compared to other countries while our economy has developed.

The choropleth map displays eight different metrics that influence destination choices. Users can switch between factors like cost of living, safety, temperature, and CO2 emissions. Each country is color-coded based on the selected metric, and hovering reveals detailed data. We added restaurant markers for major European cities to provide local context. This map enables direct comparison of potential destinations across multiple factors, making the decision-making process transparent for travelers.

For demographic analysis, we created three radar charts comparing spending patterns across socioeconomic groups, age ranges, and regions. All three use normalized scales to ensure fair comparison between categories like package holidays, restaurants, and accommodation. The color coding remains consistent to make patterns easier to spot. These charts reveal how different Danish demographics approach travel spending in an intuitive format that shows relative proportions clearly.

Finally, the correlation heatmap shows relationships between all destination factors. It uses a red-blue color scale where red indicates negative correlation and blue shows positive. All correlation values are displayed numerically for precision. This visualization provides quantitative evidence for what influences destination preferences, supporting data-driven decision making.

These four visualizations complement each other perfectly. The bubble chart establishes Denmark's global context and growth story. The choropleth map empowers users to select destinations based on their preferences. The radar charts reveal demographic insights for personalization. And the correlation heatmap validates the relationships between factors. Together, they create a comprehensive toolkit for understanding Danish tourism from historical, geographic, demographic, and analytical perspectives.

## 6. Discussion. Think critically about your creation

aggregated data. More individual behaviour

## 7. Contributions. Who did what?

### References
1. Statistics Denmark: https://www.statistikbanken.dk
2. World Bank: https://databank.worldbank.org/id/7ad54403
3. Numbeo: https://www.numbeo.com/cost-of-living
4. Global Peace Index, Temperature data and resturant data: https://www.kaggle.com/
7. CO2 Emissions: https://flightemissionmap.org

# SKal koden bare tilføjes i bunden eller hvodan gør vi det?